<a href="https://colab.research.google.com/github/teresa2005/mcPython/blob/master/10_4_cnn_%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%B6%84%EB%A5%98(%EC%A3%BC%ED%94%BC%ED%84%B0%EB%A1%9C%EB%8A%94_%EC%9E%98_%EC%95%88%EB%8F%8C%EC%95%84%EA%B0%90)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt 

In [7]:
# 테세플로우와 케라스 버젼 확인 
tf.enable_eager_execution()
print(tf.__version__)
print(keras.__version__)


1.14.0
2.2.4-tf


In [8]:
# 구글 드라이버의 데이터를 가져오기 위해서 실행  # 링크 타고 들어가 key 복사 복붙 
from google.colab import drive 
drive.mount('/gdrive',force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [17]:
# 나의 구글드라이버 (/gdrive/My Drive/)/data/dental_image/train/ 폴더이 서브폴더 확인 
!ls "/gdrive/My Drive/data/dental_image/train/"

cured  decayed	healthy


In [18]:
# 나의 구글드라이버 (/gdrive/My Drive/)/data/dental_image/test/ 폴더이 서브폴더 확인 
!ls "/gdrive/My Drive/data/dental_image/test/"

cured  decayed	healthy


In [0]:
# 학습 데이터 경로 
TRAIN_DIR="/gdrive/My Drive/data/dental_image/train/"
#테스트 데이터의 경로 
TEST_DIR="/gdrive/My Drive/data/dental_image/test/"
LR=0.001 
batch_size=32


In [0]:
# 이미지의 가로 세로
IM_WIDTH=224
IM_HEIGHT=224

In [0]:
from keras.preprocessing.image import ImageDataGenerator

# 학습에 사용될 이미지 데이터 생성기 
# epoch를 실행할때마다 변형된 이미지를 리턴해서 이미지가 많은 것 같은 효과를 줌 

train_datagen=ImageDataGenerator(
      rotation_range=180, # 회전 - 최대 80도 
      width_shift_range=0.2, # 좌우 이동 
      height_shift_range=0.2, # 상하 이동 
      horizontal_flip=True, # 좌우 반전 
      vertical_flip=True, # 상하 반전 
      )

In [0]:
# 검증에 사용될 이미지 데이터 생성기 
# 테스트에서는 이미지를 변형 할 필요가 없으므로 
# ImageDataGenerator에 값을 주지 않음 
# 이미지가 변형되지 않고 같은 이미지 리턴 
test_datagen = ImageDataGenerator()

In [31]:
# 학습에 사용될 데이터 생성기 
train_generator = train_datagen.flow_from_directory( 
                  TRAIN_DIR, # 학습데이터 경로 
                  target_size=(IM_WIDTH,IM_HEIGHT), # 위에서 정의한 이미지 사이즈 
                  batch_size=batch_size,
                  class_mode='categorical',
                  shuffle=True)

Found 150 images belonging to 3 classes.


In [32]:
# 검증에 사용될 데이터 생성기 
test_generator = test_datagen.flow_from_directory( 
                  TEST_DIR, # 학습데이터 경로 
                  target_size=(IM_WIDTH,IM_HEIGHT), # 위에서 정의한 이미지 사이즈 
                  batch_size=batch_size,
                  class_mode='categorical',
                  shuffle=True)

Found 60 images belonging to 3 classes.


In [34]:
# ImageDataGenerator 는 Target 를 지정할 필요 없이 train 폴더 안의 서브폴더가 class임 
# 클래스의 종류는 3개다. 
class_num=len(train_generator.class_indices)
class_num

3

In [35]:
# 클래스 폴더의 이름을 리턴 
custom_labels=list(train_generator.class_indices.keys())
custom_labels

['cured', 'decayed', 'healthy']

In [0]:
from tensorflow import keras

#2. 모델 구성하기 
# cnn모델 생성 
def create_model():
  # cnn 각 단계를 순서대로 저장할 객체 생성 
  model=keras.Sequential()
  model.add(keras.layers.Conv2D(filters=32,kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH,IM_HEIGHT,3)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH,IM_HEIGHT,3)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=128, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH,IM_HEIGHT,3)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH,IM_HEIGHT,3)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH,IM_HEIGHT,3)))
  model.add(keras.layers.MaxPool2D(padding='SAME'))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(1024,activation=tf.nn.relu))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(class_num,activation='softmax'))
  return model 



In [0]:
model=create_model()

In [46]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 224, 224, 32)      2432      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 64)      51264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 128)       204928    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 64)       

In [0]:
# 3.모델 학습 과정 설정하기 
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=1e-5),metrics=['accuracy'])

In [0]:
# 4. 모델 학습시키기 
# 모델 학습시 정확도와 loss를 history 에 추가 